In [1]:
import wget
import os.path
import json


import numpy as np
from numpy.linalg import det, inv
from numpy import array
import random

import ase.db
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
from gradient_super_cell import super_cell
from data_import import get_lattice_from_structure, extract_structure, get_data_as_pd

In [3]:
DATA_PATH = "database"
database_name1 = "c2db-20211702.db"
database_name2 = "c2db-20211702.db"


In [4]:
path = DATA_PATH+database_name1

options = 'is_magnetic=True, thermodynamic_stability_level=3'
props = ["formula","spgnum", "spacegroup","uid","asr_id"] 

raw_df_1 = get_data_as_pd(path,options,props)
raw_df_1 = raw_df_1[ (raw_df_1["spacegroup"]!='P1') & (raw_df_1["spacegroup"]!='Pc')& (raw_df_1["spacegroup"]!='P-1') ];

#Print
print("The number of elements is:",len(list(raw_df_1)) )
print( raw_df_1[["formula","uid"]] )

rows <generator object Database.select at 0x7fcb339122e0>
The number of elements is: 5
Empty DataFrame
Columns: [formula, uid]
Index: []


In [5]:
path = DATA_PATH+database_name2

options = 'thermodynamic_stability_level=3'
props = ["formula","spgnum", "spacegroup","gap","uid"] 

raw_df_2 = get_data_as_pd(path,options,props)
raw_df_2 = raw_df_2[ (raw_df_2["gap"]<=10e-3)& (raw_df_2["spacegroup"]!='P1') & (raw_df_2["spacegroup"]!='Pc')& (raw_df_2["spacegroup"]!='P-1') ];


#Print
print("The number of elements is:",len(list(raw_df_2)) )
print( raw_df_2[["formula","uid","gap"]] )

rows <generator object Database.select at 0x7fcb33912270>
The number of elements is: 5
Empty DataFrame
Columns: [formula, uid, gap]
Index: []


In [6]:
print("extract_data1:")
print(raw_df_1["uid"][:1])
extract_structure(raw_df_1["uid"])
print("extract_data2:")
extract_structure(raw_df_1["uid"])

extract_data1:
Series([], Name: uid, dtype: object)
extract_data2:


In [7]:
#f = "/Users/voicutu/Documents/GitHub/vdw_evolve/sandbox/XYZcolection/c2db-822.xyz"

def get_lattice_from_structure(f):
    with open(f, 'r') as file:
        json_data= json.load(file)
    mytype= "__ndarray__";
    datastruc=json_data['1']["cell"]["array"];
    if mytype in datastruc:
        shape= datastruc[mytype][0]
        dtype= datastruc[mytype][1]
        lattice = array(datastruc[mytype][2],dtype=dtype).reshape(shape)
    return lattice 


In [8]:
# Model hyperparameters

up = 99999999
bond =30
model_par = {
    
    'initialTemp': 4, 
    'finalTemp': 0.0002,

    'beta': 10,
    'bounds': [[-bond,bond] for _ in range(4)],

    'nr_neighbours': 1,
    'step_size': 3,
    'gaussian_sd':3,
    
    'gradient_influence':0.45,
    'delta_p': 0.2,
    'g_step_size':7,
    'step_size': 2,
    
    'known_min': -up,
    "start_point":[1,5,3,2],
    
    "strain_boundary":[[-0.5,0.5],[-0.5,0.5]]#[[-0.3e-14,0.3e-14],[-0.3e-14,0.3e-14]]#[[-0.3,0.3],[-0.3,0.3]]
}



In [9]:
solutions = {"uid_1":[], "uid_2":[],"new_uid":[],
             "mat_1":[], "mat_2":[],
             
             
             "tA":[],    "tB":[],
             "s_cell":[],   
             "t_cel2_no_strain":[], "diagonal_strain":[], "strain":[]
                  }


In [10]:
len(raw_df_1["uid"])

0

In [11]:
structure_path = "/Users/voicutu/Documents/GitHub/vdw_evolve/sandbox/STRUCTUREScolection"
saving_spot = 5
to_be_generated = 10
while to_be_generated!=0:
    
    data1= raw_df_1.iloc[random.randint(0,len(raw_df_1["uid"]))]
    data2= raw_df_1.iloc[random.randint(0,len(raw_df_2["uid"]))]
    uid_1 = data1["uid"]
    uid_2 = data2["uid"]
    
    new_uid =uid_1+"_"+uid_2
    
    if new_uid not in solutions["new_uid"]:
        to_be_generated = to_be_generated-1
        
        solutions["new_uid"].append(new_uid)
        solutions["uid_1"].append(uid_1)
        solutions["uid_2"].append(uid_2)
        
        # calculate the super cel 
        
        # xyz path 
        path1=structure_path+"/"+uid_1+".json"
        path2=structure_path+"/"+uid_2+".json"
        

        
        # lattices
        latice1 = get_lattice_from_structure(path1)
        latice2 = get_lattice_from_structure(path2)


        cel1 = np.array([[latice1[0][0], latice1[1][0]],
                [latice1[0][1], latice1[1][1]]])

        cel2 = np.array([[latice2[0][0], latice2[1][0]],
                [latice2[0][1], latice2[1][1]]])

        tA, tB, t_cel2_no_strain, diagonal_strain, strain = super_cell(cel1,cel2, 10, model_par)

        s_cell= np.dot(tA,cel1)
        zero = np.dot(tA,cel1)-np.dot(np.dot(strain,tB),cel2)
                         
        
        solutions["tA"].append(tA)
        solutions["tB"].append(tB)
        solutions["s_cell"].append(s_cell)
        solutions["t_cel2_no_strain"].append(t_cel2_no_strain)
        solutions["diagonal_strain"].append(diagonal_strain)
        solutions["strain"].append(diagonal_strain)
    
    # save
    if k%saving_spot==0:
        data = pd.DataFrame.from_dict(solutions)
        data.to_csv("SuperCell_"+str(len(solutions["new"])))

print("Done")

IndexError: single positional indexer is out-of-bounds